# This file creates a dataset containing venues in neighborhoods of Seattle using **Foursquare API**

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [4]:
df=pd.read_csv('seattle_zip.csv')
df.head()

,Unnamed: 0,zipcode,Latitude,Longitude
0,0,98101,47.610760,-122.336181
1,1,98102,47.621611,-122.321227
2,2,98103,47.673749,-122.343934
3,3,98104,47.600708,-122.331334
4,4,98105,47.662934,-122.320552


In [6]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,zipcode,Latitude,Longitude
0,98101,47.610760,-122.336181
1,98102,47.621611,-122.321227
2,98103,47.673749,-122.343934
3,98104,47.600708,-122.331334
4,98105,47.662934,-122.320552


#### Foursquare data

Import the file containing Foursquare account information.

In [7]:
import API_KEY
from API_KEY import CLIENT_ID,CLIENT_SECRET

In [15]:
VERSION = '20180605' # Foursquare API version
LIMIT_new=500
radius=1900 # the average size of a zip code area in seattle is 1.4 miles.
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G24RTX4QGMM2GUGHEVQHFJKKIUZC54MR0L2Q5PVWTU1FNEZL
CLIENT_SECRET:0YBQEOWRXWSHMZ0OSGV0ZPEGHW1AO5IZ5YRGK3JTB3KOA5KQ


In [16]:
def getNearbyVenues(zipcodes, latitudes, longitudes):
    
    venues_list=[]
    for zipcode, lat, lng in zip(zipcodes, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT_new)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zipcode', 
                  'lat', 
                  'long', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
zipcodes=df['zipcode']
latitudes=df['Latitude']
longitudes=df['Longitude']
seattle_venues = getNearbyVenues(zipcodes,latitudes,longitudes)

## Have a look at the dataset.

In [19]:
seattle_venues.shape

(4823, 7)

How many venues we have in each neighborhood?

In [20]:
seattle_venues['zipcode'].value_counts()

98188    100
98117    100
98109    100
98105    100
98101    100
98194    100
98174    100
98170    100
98158    100
98154    100
98138    100
98134    100
98122    100
98114    100
98106    100
98102    100
98195    100
98191    100
98127    100
98119    100
98107    100
98113    100
98190    100
98121    100
98161    100
98164    100
98144    100
98112    100
98104    100
98185    100
98181    100
98108    100
98103    100
98145    100
98133    100
98141    100
98115     94
98175     91
98126     87
98125     83
98199     81
98139     79
98165     78
98118     76
98116     70
98136     68
98166     60
98110     59
98146     58
98155     57
98177     56
98198     51
98168     23
98148     19
98178     18
98160      3
98131      3
98111      3
98129      3
98124      3
Name: zipcode, dtype: int64

Now save the dataset.

In [21]:
seattle_venues.to_csv('seattle_venues.csv')